# Imports

In [19]:
import numpy as np

from data_worker.data_worker import unpickle, unpack_data, combine_batches, split_into_batches
from torch_lib.data_worker import suit4torch
from torch_lib.Interface import Interface
from torch_lib.Nets import LargeNet, MediumNet, SmallNet

# Data

In [42]:
batches_names = [
    'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4',
    'data_batch_5'
]

data_batches = [
    unpickle(f'datasets/cifar-10-batches-py/{batch_name}') for batch_name
    in batches_names]

unpacked_batches = [
    (unpack_data(data_batch)) for data_batch
    in data_batches]

print(unpacked_batches[0][0].shape)

X, Y = combine_batches(unpacked_batches)

X_torch, Y_torch = suit4torch(X, Y)

batches = split_into_batches(X_torch, Y_torch, 3)

# torch_batches = [(suit4torch(X, Y)) for X, Y in batches]

X_batch0 = batches[0][0]
Y_batch0 = batches[0][1]

print(X.shape, Y.shape)

(10000, 32, 32, 3)
(32, 32, 3)
(50000, 32, 32, 3) (50000,)


# Train network

In [43]:
# net = SmallNet()
# net = MediumNet()
net = LargeNet()

In [53]:
net_interface = Interface(net)
net_interface.train_net(batches, 1, verbose=False)

[1,  2000] loss: 2.303
[1,  4000] loss: 2.303
[1,  6000] loss: 2.295
[1,  8000] loss: 2.184
[1, 10000] loss: 2.095
[1, 12000] loss: 2.057
[1, 14000] loss: 2.016
[1, 16000] loss: 1.961
[2, 18000] loss: 1.277
[2, 20000] loss: 1.868
[2, 22000] loss: 1.852
[2, 24000] loss: 1.805
[2, 26000] loss: 1.744
[2, 28000] loss: 1.735
[2, 30000] loss: 1.682
[2, 32000] loss: 1.668
[3, 34000] loss: 0.530
[3, 36000] loss: 1.601
[3, 38000] loss: 1.597
[3, 40000] loss: 1.584
[3, 42000] loss: 1.535
[3, 44000] loss: 1.526
[3, 46000] loss: 1.537
[3, 48000] loss: 1.518
[3, 50000] loss: 1.496
[4, 52000] loss: 1.475
[4, 54000] loss: 1.470
[4, 56000] loss: 1.472
[4, 58000] loss: 1.455
[4, 60000] loss: 1.421
[4, 62000] loss: 1.442
[4, 64000] loss: 1.432
[4, 66000] loss: 1.438
[5, 68000] loss: 0.933
[5, 70000] loss: 1.395
[5, 72000] loss: 1.425
[5, 74000] loss: 1.398
[5, 76000] loss: 1.360
[5, 78000] loss: 1.367
[5, 80000] loss: 1.382
[5, 82000] loss: 1.380


# Save weights

In [54]:
# PATH = 'saved_nets/saved_torch/small_v1.pth'
# PATH = 'saved_nets/saved_torch/medium_v1.pth'
PATH = 'saved_nets/saved_torch/large_v1.pth'

In [55]:
net_interface.save_weights(PATH)

# Predict

In [56]:
preds = net_interface.predict_net(X_batch0)
preds = preds.detach().numpy()

In [57]:
print(preds)
print(np.argmax(preds, axis=1), Y_batch0)

[[-2.9183779e+00 -1.9662402e+00  4.4520557e-01  2.6815724e+00
  -2.9322226e-04  3.1946859e-01  4.3461256e+00 -1.3899280e+00
  -1.3605685e+00 -1.0840220e+00]
 [-5.0167543e-01  8.0979884e-02 -7.8679413e-01 -7.2587729e-03
  -8.8463330e-01 -9.7887874e-01 -3.9019072e-01 -9.6788758e-01
   7.4525148e-02  2.0916569e+00]
 [-6.5086198e-01 -2.1444364e+00 -1.9671988e+00  4.7018766e-01
  -1.5143718e+00 -3.9809689e-01 -3.4725516e+00  2.2535872e+00
  -3.2368124e+00  2.7439656e+00]]
[6 9 9] tensor([6, 9, 9])


# Evaluate Accuracy

In [58]:
acc, N = net_interface.eval_acc_net(X_torch, Y_torch)
print(acc, N)

0.5302 50000


# Load weights

In [50]:
net_interface.load_weights(PATH)

In [51]:
preds = net_interface.predict_net(X_torch)
preds = preds.detach().numpy()

In [52]:
print(preds)
print(np.argmax(preds, axis=1), Y_torch)

[[ 0.01529428  0.02577884 -0.04371558 ...  0.01459533  0.0859412
   0.04300517]
 [ 0.01673645  0.0268482  -0.04321323 ...  0.01330933  0.08696277
   0.04437382]
 [ 0.01838578  0.02770798 -0.04247691 ...  0.01202005  0.08810335
   0.04617101]
 ...
 [ 0.01843791  0.02773306 -0.04254972 ...  0.01173749  0.0879827
   0.04628209]
 [ 0.01684528  0.0269551  -0.04315441 ...  0.01332093  0.08713034
   0.04446469]
 [ 0.0163019   0.02627673 -0.04331874 ...  0.01375795  0.08662
   0.04421118]]
[8 8 8 ... 8 8 8] tensor([6, 9, 9,  ..., 9, 1, 1])


# Run forward function

In [73]:
from threading import Thread
import time
import psutil


class CPUtracker(Thread):
    
    def __init__(self, Ts):
        self.Ts = Ts
        self.stopped = True
        self.values = []
        super().__init__()
    
    def start(self):
        self.stopped = False
        super().start()
    
    def stop(self):
        self.stopped = True

    def run(self):
        while not self.stopped:
            mem = psutil.virtual_memory()
            print(mem)
            print(f'{time.time():.8f}')
            time.sleep(self.Ts)
        
    

1634634624.0983648
1634634624.2068248
1634634624.3158042
1634634624.4247675
1634634624.5264096
1634634624.6429138
1634634624.7582757
1634634624.86023
1634634624.9744542
1634634625.0923345
1634634625.2076385
1634634625.3218036
1634634625.4262455
1634634625.5416522
1634634625.656925
1634634625.75855
1634634625.8757713
1634634625.981176


In [ ]:
import sys
import os
import torch
import time


def execute_net(net_interface, X_data, Y_data, batch_size, priority, loops=1, device='cuda', metrics=['ex_time', 'cpu_max', 'gpu_max', 'mem_max'], echo=True):
    if sys.platform == 'linux':
        os.nice(priority)

    device = torch.device('cpu')
    if device == 'cuda':
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        if echo and not torch.cuda.is_available():
            print("'cuda' device is not available. Using 'cpu' instead.")

    batches = split_into_batches(X_data, Y_data, batch_size)
    
    batch_times = []
    
    cpu_tracker = CPUtracker(0.1)
    cpu_tracker.start()
    
    for loop in range(loops):
        if echo:
            print('loop:', loop)
        for X_batch, Y_batch in batches:
            start_time = time.time()
            Y_pred = net_interface.predict_net(X_batch)
            batch_time = time.time() - start_time
            if echo:
                print(f'batch_time: {batch_time:.8f}')
            batch_times.append(batch_time)

    cpu_tracker.stop()

execute_net(net_interface, X_torch, Y_torch, 10, 0, 3)
    
    

1634634626.0910316
1634634626.1956158
svmem(total=8390193152, available=1268809728, percent=84.9, used=7121383424, free=1268809728)
1634634626.26711750
loop: 0
batch_time: 0.00300336
batch_time: 0.00299096
batch_time: 0.00299621
batch_time: 0.00309491
batch_time: 0.00199056
batch_time: 0.00199437
batch_time: 0.00199842
batch_time: 0.00201273
batch_time: 0.00298023
batch_time: 0.00498366
batch_time: 0.00099611
batch_time: 0.00099421
batch_time: 0.00099421
batch_time: 0.00099683
batch_time: 0.00199914
batch_time: 0.00099277
batch_time: 0.00199699
batch_time: 0.00099516
batch_time: 0.00299287
batch_time: 0.00199366
batch_time: 0.00100112
batch_time: 0.00100279
batch_time: 0.00198984
batch_time: 0.00100350
batch_time: 0.00198865
batch_time: 0.00099730
1634634626.3250675
batch_time: 0.00199485
batch_time: 0.00199294
batch_time: 0.00104618
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00

batch_time: 0.01562142
batch_time: 0.00419831
batch_time: 0.00199437
batch_time: 0.00199795
batch_time: 0.00199938
batch_time: 0.00298858
batch_time: 0.00099754
batch_time: 0.00099826
batch_time: 0.00099730
batch_time: 0.00199533
batch_time: 0.00199509
batch_time: 0.00199485
batch_time: 0.00199533
batch_time: 0.00099826
batch_time: 0.00199413
batch_time: 0.00099659
batch_time: 0.00199533
batch_time: 0.00199485
batch_time: 0.00099730
batch_time: 0.00099778
batch_time: 0.00199461
batch_time: 0.00199485
batch_time: 0.00199318
batch_time: 0.00199413
batch_time: 0.00099897
batch_time: 0.00099134
batch_time: 0.00199461
batch_time: 0.00099730
batch_time: 0.00199389
batch_time: 0.00199389
batch_time: 0.00100470
1634634627.0835366
batch_time: 0.00099874
batch_time: 0.00199509
batch_time: 0.00099826
batch_time: 0.00099659
batch_time: 0.00199485
batch_time: 0.00099754
batch_time: 0.00199294
batch_time: 0.00105906
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.0

batch_time: 0.00299239
batch_time: 0.00498819
batch_time: 0.00299263
batch_time: 0.00199223
batch_time: 0.00200176
batch_time: 0.00199723
svmem(total=8390193152, available=1256267776, percent=85.0, used=7133925376, free=1256267776)
1634634627.78011703
batch_time: 0.00598001
batch_time: 0.00199461
batch_time: 0.00200486
batch_time: 0.00198460
batch_time: 0.00299311
batch_time: 0.00199485
batch_time: 0.00499177
batch_time: 0.00498080
batch_time: 0.00199413
batch_time: 0.00199866
batch_time: 0.00403237
batch_time: 0.00197697
batch_time: 0.00199294
batch_time: 0.00299478
batch_time: 0.00199294
batch_time: 0.00199342
batch_time: 0.00099754
batch_time: 0.00498629
batch_time: 0.00199389
batch_time: 0.00199556
batch_time: 0.00199366
batch_time: 0.00099754
batch_time: 0.00199509
batch_time: 0.00199556
batch_time: 0.00299096
batch_time: 0.00199533
batch_time: 0.00099707
batch_time: 0.00199580
batch_time: 0.00099635
batch_time: 0.00199437
batch_time: 0.00099730
batch_time: 0.00199509
1634634627.8

batch_time: 0.00199437
batch_time: 0.00199628
batch_time: 0.00199413
batch_time: 0.00099683
batch_time: 0.00199437
batch_time: 0.00099730
batch_time: 0.00199533
batch_time: 0.00099659
batch_time: 0.00099850
1634634628.5236795
batch_time: 0.00299430
batch_time: 0.00298858
batch_time: 0.00199437
batch_time: 0.00199842
batch_time: 0.00298953
batch_time: 0.00199556
batch_time: 0.00299239
batch_time: 0.00298977
batch_time: 0.00299287
batch_time: 0.00199890
batch_time: 0.00198984
batch_time: 0.00199533
svmem(total=8390193152, available=1256017920, percent=85.0, used=7134175232, free=1256017920)
1634634628.55559373
batch_time: 0.00199437
batch_time: 0.00199413
batch_time: 0.00399017
batch_time: 0.00200534
batch_time: 0.00199533
batch_time: 0.00298643
batch_time: 0.00099683
batch_time: 0.00199533
batch_time: 0.00099659
batch_time: 0.00199509
batch_time: 0.00299096
batch_time: 0.00299311
batch_time: 0.00199342
batch_time: 0.00299072
batch_time: 0.00199413
batch_time: 0.00199461
batch_time: 0.00

batch_time: 0.00199890
batch_time: 0.00398493
batch_time: 0.00198913
batch_time: 0.00199962
batch_time: 0.00198388
batch_time: 0.00199389
batch_time: 0.00199652
batch_time: 0.00301719
batch_time: 0.00099730
batch_time: 0.00100017
1634634629.2830822
batch_time: 0.00099897
batch_time: 0.00199223
batch_time: 0.00298905
batch_time: 0.00199723
batch_time: 0.00299263
batch_time: 0.00299263
batch_time: 0.00399017
batch_time: 0.00199413
batch_time: 0.00099683
batch_time: 0.00099659
batch_time: 0.00199342
batch_time: 0.00296879
svmem(total=8390193152, available=1256525824, percent=85.0, used=7133667328, free=1256525824)
1634634629.31499720
batch_time: 0.00299287
batch_time: 0.00099683
batch_time: 0.00199509
batch_time: 0.00099778
batch_time: 0.00099754
batch_time: 0.00199485
batch_time: 0.00300241
batch_time: 0.00199604
batch_time: 0.00199723
batch_time: 0.00199413
batch_time: 0.00199389
batch_time: 0.00199461
batch_time: 0.00303531
batch_time: 0.00395775
batch_time: 0.00198555
batch_time: 0.00

...

batch_time: 0.00000000
batch_time: 0.00715566
batch_time: 0.00299025
batch_time: 0.00199342
1634634656.0957222batch_time: 0.00199628

batch_time: 0.00299096
batch_time: 0.00199580
batch_time: 0.00199461
batch_time: 0.00399137
batch_time: 0.00298977
batch_time: 0.00199556
batch_time: 0.00299788
batch_time: 0.00298548
batch_time: 0.00199437
batch_time: 0.00299239
batch_time: 0.00199461
batch_time: 0.00299239
batch_time: 0.00299954
batch_time: 0.00198841
batch_time: 0.00100374
batch_time: 0.00099802
batch_time: 0.00199366
batch_time: 0.00199533
batch_time: 0.00199437
batch_time: 0.00199485
batch_time: 0.00199461
batch_time: 0.00200605
batch_time: 0.00198269
batch_time: 0.00100255
batch_time: 0.00100493
batch_time: 0.00198674
batch_time: 0.00199461
batch_time: 0.00198197
batch_time: 0.00199461
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
svmem(total=839

batch_time: 0.00199080
batch_time: 0.00299287
batch_time: 0.00299072
batch_time: 0.00199461
batch_time: 0.00199485
batch_time: 0.00299191
batch_time: 0.00299382
batch_time: 0.00399065
batch_time: 0.00299406
batch_time: 0.00298691
batch_time: 0.00299239
batch_time: 0.00199866
batch_time: 0.00199199
svmem(total=8390193152, available=1247952896, percent=85.1, used=7142240256, free=1247952896)
1634634657.05043650
batch_time: 0.00498581
batch_time: 0.00199389
batch_time: 0.00199270
batch_time: 0.00199437
batch_time: 0.00199699
1634634657.0643969
batch_time: 0.00399017
batch_time: 0.00299120
batch_time: 0.00199437
batch_time: 0.00199437
batch_time: 0.00199461
batch_time: 0.00199652
batch_time: 0.00299478
batch_time: 0.00199175
batch_time: 0.00100231
batch_time: 0.00099730
batch_time: 0.00199485
batch_time: 0.00199437
batch_time: 0.00199604
batch_time: 0.00200653
batch_time: 0.00297737
batch_time: 0.00199509
batch_time: 0.00199485
batch_time: 0.00199461
batch_time: 0.00199509
batch_time: 0.00

batch_time: 0.00000000
batch_time: 0.00679255
batch_time: 0.00108409
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.01562119
batch_time: 0.00582290
batch_time: 0.00221443
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00997901
batch_time: 0.00199556
batch_time: 0.00299191
batch_time: 0.00299168
batch_time: 0.00199485
batch_time: 0.00199509
batch_time: 0.00302100
batch_time: 0.00196528
batch_time: 0.00199509
batch_time: 0.00199628
batch_time: 0.00199246
batch_time: 0.00099754
batch_time: 0.00099778
svmem(total=8390193152, available=1247358976, percent=85.1, used=7142834176, free=1247358976)
1634634657.82804394
batch_time: 0.00299048
batch_time: 0.00199461
batch_time: 0.00199509
batch_time: 0.00199389
batch_time: 0.00199914
batch_time: 0.00199199
batch_time: 0.00199389
batch_time: 0.00100350
1634634657.844
batch_time: 0.00298500
batch_time: 0.002000

batch_time: 0.01561880
batch_time: 0.00534654
batch_time: 0.00199175
batch_time: 0.00199676
batch_time: 0.00299263
batch_time: 0.00199580
batch_time: 0.00398660
batch_time: 0.00199366
batch_time: 0.00299501
batch_time: 0.00199056
batch_time: 0.00199533
batch_time: 0.00300097
batch_time: 0.00298667
batch_time: 0.00099659
batch_time: 0.00099730
batch_time: 0.00199461
batch_time: 0.00199437
batch_time: 0.00199580
batch_time: 0.00199318
batch_time: 0.00099778
batch_time: 0.00398684
batch_time: 0.00199509
batch_time: 0.00199509
batch_time: 0.00199437
batch_time: 0.00199533
batch_time: 0.00199485
batch_time: 0.00199461
batch_time: 0.00108838
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
svmem(total=8390193152, available=1247162368, percent=85.1, used=7143030784, free=1247162368)1634634658.5958016

1634634658.59580159
batch_time: 0.01562333
batch_time: 0.00000000
batch_time: 0.00

batch_time: 0.00200081
batch_time: 0.00298810
batch_time: 0.00199389
batch_time: 0.00299525
batch_time: 0.00199199
batch_time: 0.00398540
batch_time: 0.00299144
batch_time: 0.00199485
batch_time: 0.00299239
batch_time: 0.00299239
batch_time: 0.00199342
batch_time: 0.00299311
batch_time: 0.00299120
batch_time: 0.00099707
batch_time: 0.00099826
batch_time: 0.00199509
batch_time: 0.00199366
batch_time: 0.00199533
batch_time: 0.00199413
batch_time: 0.00200248
batch_time: 0.00199723
batch_time: 0.00299144
batch_time: 0.00199270
batch_time: 0.00299263
batch_time: 0.00199342
batch_time: 0.00313687
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
batch_time: 0.00000000
svmem(total=8390193152, available=1248419840, percent=85.1, used=7141773312, free=1248419840)1634634659.3454492

1634634659.34544921
batch_time: 0.01768422
batch_time: 0.00199389
batch_time: 0.00199509
batch_time: 0.00199628
batch_time: 0.00199437
batch_time: 0.00199986
batch_time: 0.00